# Training VAE for RGB Images

### Imports

In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedDataset

from models import FC_VAE
from train_vae import train, evaluate


### SETUP

In [ ]:
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 10
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

features_file = "saved_features/saved_feat_I3D_25_dense_D1"

### TRAINING

In [ ]:
train_dataset = FeaturesExtendedDataset(features_file,'train')
train_loader_rgb = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)


model = FC_VAE(dim_input=1024, nz=64)
model.to(DEVICE)
print(f'Initial model device: {model.device}')

# Create Optimizer & Scheduler objects
optimizer = Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train(model, optimizer, EPOCHS, DEVICE, train_loader_rgb, train_loader_rgb, BATCH_SIZE, scheduler)

torch.save(model.state_dict(), f'./saved_models/VAE_RGB/final_VAE_RGB_epoch_{EPOCHS}.pth')


### EVALUATION

In [ ]:
train_dataset = FeaturesExtendedDataset(features_file,'train')
test_dataset = FeaturesExtendedDataset(features_file,'test')
train_loader_rgb = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_loader_rgb = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

model = FC_VAE(dim_input=1024, nz=64)
model.to(DEVICE)
model.load_state_dict(torch.load(f'./saved_models/VAE_RGB/final_VAE_RGB_epoch_50.pth'))

reconstructed, originals = evaluate(model, DEVICE, train_loader_rgb)
reconstructed2, originals2 = evaluate(model, DEVICE, test_loader_rgb)

# Training VAE for EMG Signals

In [12]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedEMGDataset

from models import FC_VAE
from train_vae import train, evaluate
import pandas as pd
import numpy as np

### SETUP

In [46]:
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 10
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

LSTM_features_file_train = "saved_features/EMG_Emb_LSTM_25_dense_D1_train.pkl"
LSTM_features_file_test = "saved_features/EMG_Emb_LSTM_25_dense_D1_test.pkl"
STAT_features_file_train = "saved_features/EMG_Emb_Stat_25_dense_D1_train.pkl"
STAT_features_file_test = "saved_features/EMG_Emb_Stat_25_dense_D1_test.pkl"

------ USING APPLE SILICON GPU ------


### TRAINING

In [48]:
train_dataset = FeaturesExtendedEMGDataset(LSTM_features_file_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True, multiprocessing_context='fork' if torch.backends.mps.is_available() else None)

for i in train_loader:
    print(i["features"].shape)
    break

torch.Size([32, 112])


In [ ]:
model = FC_VAE(dim_input=1024, nz=64)
model.to(DEVICE)
print(f'Initial model device: {model.device}')

# Create Optimizer & Scheduler objects
optimizer = Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train(model, optimizer, EPOCHS, DEVICE, train_loader_rgb, train_loader_rgb, BATCH_SIZE, scheduler)

torch.save(model.state_dict(), f'./saved_models/VAE_EMG_LSTM/final_VAE_EMG_LSTM_epoch_{EPOCHS}.pth')

### EVALUATION

In [ ]:
train_dataset = FeaturesExtendedEMGDataset(features_file)
test_dataset = FeaturesExtendedDataset(features_file_test)
train_loader_emg = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_loader_emg = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

model = FC_VAE(dim_input=1024, nz=64)
model.to(DEVICE)
model.load_state_dict(torch.load(f'./saved_models/VAE_EMG/final_VAE_EMG_epoch_50.pth'))

reconstructed, originals = evaluate(model, DEVICE, train_loader_rgb)
reconstructed2, originals2 = evaluate(model, DEVICE, test_loader_rgb)

# Fine Tuning Training [ RGB --> EMG ]

### IMPORTS

In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedDataset

from models import FC_VAE
from train_vae import train, evaluate

### SETUP

In [ ]:
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 10
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

features_file_rgb = ""
features_file_emg = ""


### TRAINING

In [ ]:
train_dataset_rgb = FeaturesExtendedDataset(features_file_rgb,'train')
train_loader_rgb = DataLoader(train_dataset_rgb, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

train_dataset_emg = FeaturesExtendedDataset(features_file_emg,'train')
train_loader_emg = DataLoader(train_dataset_emg, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)


model_finetune = FC_VAE(dim_input=1024, nz=64)
model_finetune.to(DEVICE)

# Carica i pesi del modello RGB per l'encoder
checkpoint_rgb = torch.load('./saved_models/VAE_RGB/final_VAE_RGB_epoch_50.pth', map_location=DEVICE)
model_finetune.encoder.load_state_dict({k: v for k, v in checkpoint_rgb.items() if 'encoder' in k})

# Carica i pesi del modello EMG per il decoder
checkpoint_emg = torch.load('./saved_models/VAE_EMG/final_VAE_EMG_epoch_50.pth', map_location=DEVICE)
model_finetune.decoder.load_state_dict({k: v for k, v in checkpoint_emg.items() if 'decoder' in k})

# Create Optimizer & Scheduler objects
optimizer = Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)


train(model, optimizer, EPOCHS, DEVICE, train_loader_rgb, train_loader_emg, BATCH_SIZE, scheduler)

torch.save(model.state_dict(), f'./saved_models/VAE_RGB/final_VAE_RGB_epoch_{EPOCHS}.pth')
